In [103]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

"""
Copied starter code from Andrej, because I totally understand it & I'm not rewriting this stuff for the third time XD
"""

# read in all the words
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

# ok biolerplate done, now we get to the action:

# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

batch_size = 32
n = batch_size # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()  # retain_grad enables .grad attribute for all nodes (not only for leaves)
loss.backward()
loss


32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']
{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27
torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])
4137


In [104]:
"""dlogprobs reasoning process"""
# dlogprobs is how dlogprobs affects loss
# this is how loss depends on dlogprobs: loss = -logprobs[range(n), Yb].mean()
# It's good to think of simpler examples.
# mean is just a sum of elements over number of them
# -(a + b + c) / 3 = -1/3*a -1/3*b -1/3*c
# dloss/da = -1/3 or -1/n in general form
# However, not all elements of logprobs affect loss, so it doesn't matter how we set them. Their grad is 0

"""dprobs reasoning process"""
# Okay, it's how probs affects loss. probs affects loss how logprobs affect loss times how probs affects logprobs
# Shape remains the same. logprobs = probs.log()
# Well, google answer to derivative of log base e w.r.t input is 1 / input
# It appeared to be right, but honestly idk how (I'll use wolfram alpha if needed)

"""counts_sum_inv, counts_sum and counts reasoning process"""
# I thought it's simple, but shapes are different
# simpler example is (3, 3) and (3, 1)
# It means that (3,1) tensor is replicated 3 times horizontally
# a11 * b1 a12 * b1 a13 * b1
# a21 * b2 a22 * b2 a23 * b2
# a31 * b3 a32 * b3 a33 * b3

# So simple outgrad * other is grad for replicated tensor
# As there are multiple gradients for each of bs we sum them horizontally (sum over row)
# Got it! Got caught here!
# Also it's too early to celebrate, counts appears twice and counts_sum_inv depends on counts_sum, which depends on counts
# Counts_sum derivative is pretty simple (power rule), but counts derivative is more interesting
# We do horizontal summation, so what we do is simply turning

# a11 a12 a13      (a11+a12+a13)
# a21 a22 a23  ==> (a21+a22+a23)
# a31 a32 a33      (a31+a32+a33)
# For each of the elements inner derivative is just one.
# So we can represent it as torch.ones (actually ones_like to avoid harcoded numbers)
# And multiplication by outer grad works well, since broadcasting is applied horizontally
# Since it's the 2nd branch for counts we add this grad (not reassign)
"""norm logits reasoning process"""
# Counts is norm_logits with elementwise .exp() applied
# Easy!

'norm logits reasoning process'

In [105]:
logits.shape, logit_maxes.shape
print(hprebn.shape, bnmeani.shape, bndiff.shape)

torch.Size([32, 64]) torch.Size([1, 64]) torch.Size([32, 64])


In [107]:
# Exercise 1: backprop through the whole thing manually, 
# backpropagating through exactly all of the variables 
# as they are defined in the forward pass above, one by one


dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0/n
cmp('logprobs', dlogprobs, logprobs)  # Correct!
dprobs = dlogprobs * 1.0 / probs
cmp('probs', dprobs, probs)  # Correct!
dcounts_sum_inv = (dprobs * counts).sum(dim=1, keepdim=True)
dcounts = dprobs * counts_sum_inv  # broadcasting's just fine here.
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
dcounts_sum = (-counts_sum**-2) * dcounts_sum_inv
cmp('counts_sum', dcounts_sum, counts_sum)
dcounts += torch.ones_like(dcounts) * dcounts_sum  # Broadcasting does the thing
cmp('counts', dcounts, counts)
dnorm_logits = dcounts * norm_logits.exp()
cmp('norm_logits', dnorm_logits, norm_logits)
dlogit_maxes = -dnorm_logits.sum(dim=1, keepdim=True)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
dlogits = dnorm_logits.clone()
# Second branch for dlogits
dlogits += F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogit_maxes
cmp('logits', dlogits, logits)
dh = dlogits @ W2.T
cmp('h', dh, h)
dW2 = h.T @ dlogits
cmp('W2', dW2, W2)
db2 = dlogits.sum(dim=0)   # Broadcasted vertically (32, 27) + (27,) -> 32 copies added
cmp('b2', db2, b2)
dhpreact = (1.0 - h**2) * dh
# From here, for some reason it's not exactly correct (maybe due to FLOPS)
# Nothing to worry about, but just sadge ;(
cmp('hpreact', dhpreact, hpreact)
dbngain = (dhpreact * bnraw).sum(0, keepdim=True)
cmp('bngain', dbngain, bngain)
dbnbias = dhpreact.sum(0, keepdim=True)
cmp('bnbias', dbnbias, bnbias)
dbnraw = dhpreact * bngain  # Broadcasting takes care
cmp('bnraw', dbnraw, bnraw)
dbnvar_inv = (dbnraw * bndiff).sum(0, keepdim=True)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
dbnvar = (-0.5 * (bnvar + 1e-5)**-1.5) * dbnvar_inv
cmp('bnvar', dbnvar, bnvar)
dbndiff2 = torch.ones_like(bndiff2) * (1/(n-1)) * dbnvar
cmp('bndiff2', dbndiff2, bndiff2)
dbndiff = (dbnraw * bnvar_inv)  # First branch
dbndiff += 2 * bndiff * dbndiff2  # Second branch
cmp('bndiff', dbndiff, bndiff)
dhprebn = dbndiff.clone()
dbnmeani = -dbndiff.sum(0, keepdim=True)
cmp('bnmeani', dbnmeani, bnmeani)
dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)
cmp('hprebn', dhprebn, hprebn)
dembcat = dhprebn @ W1.T
cmp('embcat', dembcat, embcat)
dW1 = embcat.T @ dhprebn
cmp('W1', dW1, W1)
db1 = dhprebn.sum(0)
cmp('b1', db1, b1)
demb = dembcat.view(emb.shape)
cmp('emb', demb, emb)
dC = torch.zeros_like(C)
for i in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        ix = Xb[i, j]
        dC[ix] += demb[i, j]
cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: False | approximate: True  | maxdiff: 4.656612873077393e-10
bngain          | exact: False | approximate: True  | maxdiff: 1.862645149230957e-09
bnbias          | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09
bnraw   

## Despite some silly moves from floating point arithmetics, which ruined beauty of my work it's done!
## "Beast" is backpropagated!

---

# Ex. 2 - Cross Entropy Loss backward with less work done
(Not today tho, today I'm more than okay with beast backprop as a final piece!)

In [108]:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), "diff:", (loss_fast - loss).item())

3.3648874759674072 diff: 0.0
